In [12]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pprint import pprint
from citipy import citipy
import requests
import time
import datetime

# weather.io API Key
#from config import api_key

In [13]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


635

In [14]:
cities_df = pd.DataFrame(cities)
#cities_df.rename({1:"City"},inplace=True)
cities_df["lat"]=-99
cities_df["temp"]=-99
cities_df["sunrise"]=-99
cities_df["sunset"]=-99
cities_df["solar_rad"]=-99
#cities_df["elev_angle"]=-99
cities_df["day_length"]=-99
cities_df.head()

,0,lat,temp,sunrise,sunset,solar_rad,day_length
0,oiso,-99,-99,-99,-99,-99,-99
1,mahebourg,-99,-99,-99,-99,-99,-99
2,khormuj,-99,-99,-99,-99,-99,-99
3,palmer,-99,-99,-99,-99,-99,-99
4,pokhara,-99,-99,-99,-99,-99,-99


In [15]:
def timeDiff(sunrise, sunset):
# some stations have a sunrise and sunset of 0 because the sun is always above
# or below the horizon
   # if (sunrise > 0 and sunset > 0):
        
#  Convert the integer sunrise and sunset to actual date-time variables
        #rise = datetime.datetime.fromtimestamp(sunrise)
        #set = datetime.datetime.fromtimestamp(sunset)
        rise = sunrise
        sset = sunset
        fmt = '%Y-%m-%d %H:%M:%S'
               
#  This converts to date/time objects
        d1 = datetime.datetime.strptime(rise, fmt)
        d2 = datetime.datetime.strptime(sset, fmt)
        
#  compute the number of minutes between the two dates
        diff = d2 - d1
        diff = (diff.days * 24 * 60) + (diff.seconds/60)
        #diff = ((sunset - sunrise)/60)
        if(diff < 0):
            diff = 1440 + diff
        
        return (diff)
          #return ((diff.days * 24 * 60) + (diff.seconds/60))
    
   # else:
     #   return (0)


In [19]:

for i,city in enumerate(cities):
    url = f"https://api.weatherbit.io/v2.0/current?city={city}&key=70361cbf92284b03bfbfaac344ac49ef"
    #print(url)
    
    try:
        
        city_data = requests.get(url).json()
        
        lat = city_data["data"][0]["lat"]
        temp = city_data["data"][0]["temp"]
        solar_rad = city_data["data"][0]["solar_rad"]
        elev_angle = city_data["data"][0]["elev_angle"]
        sunrise = city_data["data"][0]["sunrise"]
        sunset = city_data["data"][0]["sunset"]
        ob_time = city_data["data"][0]["ob_time"]
        
        # "ob_time":"2017-08-28 16:45"
        
        hold = ob_time.split(" ")
        ob_time = hold[0]
             
        sunrise = f"{ob_time} {sunrise}:00" 
        sunset = f"{ob_time} {sunset}:00"
        
        print(sunrise, sunset)
        print(sunset)
        
        minutes = timeDiff(sunrise, sunset)         
        
        cities_df.iloc[i, cities_df.columns.get_loc("lat")] = lat
        cities_df.iloc[i, cities_df.columns.get_loc("temp")] = temp
        cities_df.iloc[i, cities_df.columns.get_loc("sunrise")] = sunrise
        cities_df.iloc[i, cities_df.columns.get_loc("sunset")] = sunset
        cities_df.iloc[i, cities_df.columns.get_loc("solar_rad")] = solar_rad
        cities_df.iloc[i, cities_df.columns.get_loc("day_length")] = minutes
        
    except:
        print("missed " + city)
        
#cities_df.head()

cities_df.to_csv('cities.csv', index=False)

2020-10-27 20:59:00 2020-10-27 07:52:00
2020-10-27 07:52:00
2020-10-27 01:30:00 2020-10-27 14:16:00
2020-10-27 14:16:00
2020-10-27 02:39:00 2020-10-27 13:47:00
2020-10-27 13:47:00
2020-10-27 10:49:00 2020-10-27 22:36:00
2020-10-27 22:36:00
2020-10-27 00:25:00 2020-10-27 11:42:00
2020-10-27 11:42:00
2020-10-27 02:56:00 2020-10-27 14:57:00
2020-10-27 14:57:00
2020-10-27 02:54:00 2020-10-27 12:42:00
2020-10-27 12:42:00
2020-10-27 21:57:00 2020-10-27 10:11:00
2020-10-27 10:11:00
2020-10-27 09:58:00 2020-10-27 21:43:00
2020-10-27 21:43:00
2020-10-27 05:29:00 2020-10-27 16:40:00
2020-10-27 16:40:00
2020-10-27 10:49:00 2020-10-27 23:11:00
2020-10-27 23:11:00
2020-10-27 08:53:00 2020-10-27 23:41:00
2020-10-27 23:41:00
2020-10-27 04:55:00 2020-10-27 15:21:00
2020-10-27 15:21:00
2020-10-27 02:13:00 2020-10-27 15:07:00
2020-10-27 15:07:00
2020-10-27 05:40:00 2020-10-27 17:46:00
2020-10-27 17:46:00
2020-10-27 11:25:00 2020-10-27 18:48:00
2020-10-27 18:48:00
2020-10-27 09:09:00 2020-10-27 23:46:00


2020-10-27 09:51:00 2020-10-27 23:20:00
2020-10-27 23:20:00
2020-10-27 20:20:00 2020-10-27 09:15:00
2020-10-27 09:15:00
2020-10-27 11:24:00 2020-10-27 22:12:00
2020-10-27 22:12:00
2020-10-27 21:55:00 2020-10-27 09:28:00
2020-10-27 09:28:00
2020-10-27 23:26:00 2020-10-27 10:08:00
2020-10-27 10:08:00
2020-10-27 16:54:00 2020-10-27 05:28:00
2020-10-27 05:28:00
2020-10-27 09:50:00 2020-10-27 21:33:00
2020-10-27 21:33:00
2020-10-27 09:58:00 2020-10-27 21:43:00
2020-10-27 21:43:00
missed airai
2020-10-27 06:28:00 2020-10-27 14:15:00
2020-10-27 14:15:00
2020-10-27 06:58:00 2020-10-27 17:28:00
2020-10-27 17:28:00
2020-10-27 10:39:00 2020-10-27 22:33:00
2020-10-27 22:33:00
2020-10-27 10:41:00 2020-10-27 22:39:00
2020-10-27 22:39:00
2020-10-27 13:55:00 2020-10-27 00:48:00
2020-10-27 00:48:00
missed azanja
2020-10-27 09:03:00 2020-10-27 22:36:00
2020-10-27 22:36:00
2020-10-27 00:12:00 2020-10-27 09:42:00
2020-10-27 09:42:00
2020-10-27 02:56:00 2020-10-27 14:57:00
2020-10-27 14:57:00
2020-10-27 23

2020-10-27 08:36:00 2020-10-27 21:14:00
2020-10-27 21:14:00
2020-10-27 22:35:00 2020-10-27 09:46:00
2020-10-27 09:46:00
2020-10-27 09:13:00 2020-10-27 23:14:00
2020-10-27 23:14:00
2020-10-27 16:56:00 2020-10-27 00:14:00
2020-10-27 00:14:00
2020-10-27 21:38:00 2020-10-27 07:48:00
2020-10-27 07:48:00
2020-10-27 17:28:00 2020-10-27 07:25:00
2020-10-27 07:25:00
2020-10-27 12:52:00 2020-10-27 23:37:00
2020-10-27 23:37:00
2020-10-27 23:26:00 2020-10-27 10:08:00
2020-10-27 10:08:00
2020-10-27 21:38:00 2020-10-27 07:48:00
2020-10-27 07:48:00
2020-10-27 22:49:00 2020-10-27 09:50:00
2020-10-27 09:50:00
2020-10-27 16:54:00 2020-10-27 05:28:00
2020-10-27 05:28:00
2020-10-27 11:26:00 2020-10-27 22:42:00
2020-10-27 22:42:00
2020-10-27 16:33:00 2020-10-27 03:59:00
2020-10-27 03:59:00
2020-10-27 21:34:00 2020-10-27 08:34:00
2020-10-27 08:34:00
2020-10-27 19:59:00 2020-10-27 09:12:00
2020-10-27 09:12:00
2020-10-27 12:15:00 2020-10-27 23:22:00
2020-10-27 23:22:00
2020-10-27 03:12:00 2020-10-27 15:21:00


2020-10-27 05:53:00 2020-10-27 16:19:00
2020-10-27 16:19:00
2020-10-27 04:22:00 2020-10-27 15:26:00
2020-10-27 15:26:00
2020-10-27 06:28:00 2020-10-27 17:18:00
2020-10-27 17:18:00
2020-10-27 05:10:00 2020-10-27 15:34:00
2020-10-27 15:34:00
2020-10-27 00:48:00 2020-10-27 10:20:00
2020-10-27 10:20:00
2020-10-27 00:35:00 2020-10-27 12:22:00
2020-10-27 12:22:00
2020-10-27 14:53:00 2020-10-27 03:47:00
2020-10-27 03:47:00
2020-10-27 21:29:00 2020-10-27 09:21:00
2020-10-27 09:21:00
2020-10-27 10:16:00 2020-10-27 19:13:00
2020-10-27 19:13:00
2020-10-27 23:13:00 2020-10-27 10:32:00
2020-10-27 10:32:00
2020-10-27 22:43:00 2020-10-27 10:30:00
2020-10-27 10:30:00
2020-10-27 10:33:00 2020-10-27 20:42:00
2020-10-27 20:42:00
2020-10-27 00:40:00 2020-10-27 12:31:00
2020-10-27 12:31:00
2020-10-27 11:11:00 2020-10-27 21:41:00
2020-10-27 21:41:00
2020-10-27 11:27:00 2020-10-27 21:48:00
2020-10-27 21:48:00
2020-10-27 04:39:00 2020-10-27 14:56:00
2020-10-27 14:56:00
2020-10-27 16:54:00 2020-10-27 00:36:00


2020-10-27 08:36:00 2020-10-27 21:14:00
2020-10-27 21:14:00
2020-10-27 06:00:00 2020-10-27 16:25:00
2020-10-27 16:25:00
2020-10-27 03:55:00 2020-10-27 14:19:00
2020-10-27 14:19:00
2020-10-27 21:46:00 2020-10-27 10:10:00
2020-10-27 10:10:00
2020-10-27 06:05:00 2020-10-27 13:21:00
2020-10-27 13:21:00
2020-10-27 18:56:00 2020-10-27 08:03:00
2020-10-27 08:03:00
2020-10-27 02:13:00 2020-10-27 15:07:00
2020-10-27 15:07:00
missed mrirt
2020-10-27 23:15:00 2020-10-27 09:32:00
2020-10-27 09:32:00
2020-10-27 02:56:00 2020-10-27 14:57:00
2020-10-27 14:57:00
2020-10-27 08:11:00 2020-10-27 20:25:00
2020-10-27 20:25:00
2020-10-27 21:41:00 2020-10-27 08:10:00
2020-10-27 08:10:00
2020-10-27 19:40:00 2020-10-27 08:20:00
2020-10-27 08:20:00
2020-10-27 02:19:00 2020-10-27 13:55:00
2020-10-27 13:55:00
2020-10-27 03:47:00 2020-10-27 16:33:00
2020-10-27 16:33:00
2020-10-27 03:03:00 2020-10-27 16:03:00
2020-10-27 16:03:00
2020-10-27 21:41:00 2020-10-27 08:10:00
2020-10-27 08:10:00
2020-10-27 08:56:00 2020-10